Here we have a mathematical equation to understand the backpropagation process. The equation is inspired from the activation process of a Neural Network.
Let's take an example of a Neuron in a Neural Network fed with 3 input parameters (x0, x1 and x2).

Now every Neuron will have weights assigned to these input parameters which will be tuned as our model trains and learns the input dataset patterns.
In my previous article, I have talked about this process in detail. The first process is to calculate the output of the Neuron and this process is called
Forward Pass or Forward Propagation.

The calculation of the output of Neuron happens as follows:


y = w0.x0 + w1.x1 + w2.x2 + b
output = activation_function(y)

The result if fed into an activation function to get the final output of the neuron. We will consider the first part of the above calculation to understand
the backpropagation process.

In [7]:
import math

class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0
        self.label = label
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh (Activation Function)')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"


In [12]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ %s | data: %.4f }" % (n.label, n.data), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [10]:
w0 = Value(1.3, label="weight: w0")
x0 = Value(1.6, label="input: x0")

w1 = Value(1.2, label="weight: w1")
x1 = Value(0.8, label="input: x1")

w2 = Value(1.1, label="weight: w2")
x2 = Value(0.5, label="input: x2")

b = Value(0.2, label="bias: b")

w0x0 = w0 * x0
w0x0.label = "w0.x0"
w1x1 = w1 * x1
w1x1.label = "w1.x1"
w2x2 = w2 * x2
w2x2.label = "w2.x2"

w0x0w1x1 = w0x0 + w1x1
w0x0w1x1.label = "w0x0 + w1x1"

w0x0w1x1w2x2 = w0x0w1x1 + w2x2
w0x0w1x1w2x2.label = "w0x0 + w1x1 + w2x2"

z = w0x0w1x1w2x2 + b
z.label = "z"

y = z.tanh()
y.label = "y (final output of Neuron)"

y

Value(data=0.9989793986334531, grad=0)

In [13]:
dot = draw_dot(y, rankdir='TB')
dot.render('gout_forward_pass')

'gout_forward_pass.svg'

The above flowchart depicts the above mathemtical equation for calculating y. This graph representation of the equation will further help us in
understanding the Backpropagation process.

As our next step we will bacpropagate though the above equation and calculate the gradient/derivative of each terms with respect to the final value y.

In [32]:
y.backward()